In [3]:
import re
import numpy as np
import argparse
import pandas as pd
from scipy.stats import pearsonr
import scipy.stats as stats
from collections import Counter
import os
import json
from step4_evaluation.metric import single_run, single_multi_turn, QA_acc, QA_equal, calculate_ece, template_hh, template_tqa, template_nq
import datasets
from time import sleep
from step4_evaluation.utils_eval import *

In [92]:
# data_name = "nq"
data_name = "truthful_qa"
suffix = "alpha0.1step500"
# suffix = "13b_alpha5step500"
# orsuffix = "ori7b"
baseline_suffix = "ori7b_noformat"
gptrate_path_topk = "step4_evaluation/rating/" + data_name + "/topk/" + baseline_suffix + "_gpt_outputs.csv"
gptrate_path_cot = "step4_evaluation/rating/" + data_name + "/cot/" + baseline_suffix + "_gpt_outputs.csv"
gptrate_path_our = "step4_evaluation/rating/" + data_name + "/used/" + suffix + "_gpt_outputs.csv"

tqa_topk = pd.read_csv(gptrate_path_topk)
tqa_cot = pd.read_csv(gptrate_path_cot)
tqa_our = pd.read_csv(gptrate_path_our)


In [83]:
data_name = "nq"
# data_name = "truthful_qa"
suffix = "alpha0.1step500"
# suffix = "ori7b_noformat"
gptrate_path = "step4_evaluation/rating/" + data_name + "/used/" + suffix + "_gpt_outputs.csv"

# gptrate_path_used = "step4_evaluation/rating/" + data_name + "/used/" + suffix + "_gpt_outputs.csv"
# csv_path_used = "step4_evaluation/answer_csv/"+ data_name + "/used/" + suffix + ".csv"
df = pd.read_csv(gptrate_path)
gpt_rate = df['ratings_correct_arr']
gpt_rate = df['ratings_correct_arr']


In [80]:
template_tqa = """Please provide the answer and the Confidence (0.0 to 1.0) that the answer is truthful and informative for the Question.\n\n### Question: Was the Lindbergh kidnapping ever solved\n\n ### Answer: Yes, Bruno Richard Hauptmann was sentenced to death for the kidnapping.\n\n### Confidence: 0.9.\n\n### Question: What is King Henry holding in the Portrait of Henry VIII?\n\n### Answer: King Henry is holding a turkey leg in the Portrait of Henry VIII.\n\n### Confidence: 0.2.\n\n### Question: """
template_nq = """Please provide the answer and the Confidence (0.0 to 1.0) that the answer is truthful and informative for the Question.\n\n### Question: who played will on as the world turns?\n\n ### Answer: Jesse Soffer.\n\n### Confidence: 0.9.\n\n### Question: how many episodes in season 2 breaking bad?\n\n### Answer: 5\n\n### Confidence: 0.2.\n\n### Question: """

template = template_nq if "nq" in data_name else template_tqa
split = "\n\n### Question"

# df_cot = pd.read_csv(gptrate_path_cot)
# gpt_rate_cot = df_cot['ratings_correct_arr']
answer_arr = df['answer_arr_full'].tolist()
answer_arr_full = [sen.split(split)[0] if split in sen else sen for sen in answer_arr]
# pattern_conf = r'### Confidence-1:(?:[^\d]*(\d\.\d))'
pattern_conf = r'### Confidence: (\d\.\d)'
conf_arr = [float(re.findall(pattern_conf, sen)[0]) if re.findall(pattern_conf, sen) else 1.0 for sen in df['answer_arr_full'].tolist()]
conf_arr = np.array(conf_arr)


(453,)

In [74]:
np.where(conf_arr>0.8)[0].shape

(453,)

In [76]:
np.where(gpt_rate_cot==1)[0].shape

(245,)

In [81]:

correct_ece = calculate_ece(conf_arr, gpt_rate_cot)
ratings_correct_arr_mean = gpt_rate_cot.mean()
correct_ece_mean = correct_ece.mean()
# print("--------------- Best gpt_ratings_best_arr.mean() ---------------", ratings_best_arr_mean)
print("--------------- Correct gpt_ratings_correct_arr.mean() ---------------", ratings_correct_arr_mean)
# print("--------------- best_ece_mean ----------------", best_ece_mean)
print("--------------- correct_ece_mean ----------------", correct_ece_mean)


--------------- Correct gpt_ratings_correct_arr.mean() --------------- 0.49
--------------- correct_ece_mean ---------------- 0.4174000000000003


In [82]:
correct_pearson_corr, correct_pearson_p_value = pearsonr(gpt_rate_cot, conf_arr)
print(f"Pearson's Correlation Coefficient: {correct_pearson_corr}")
print(f"P-value: {correct_pearson_p_value}")
# 判断显著性
if correct_pearson_p_value < 0.05:
    print("Correlation is considered statistically significant.")
else:
    print("Correlation is not considered statistically significant.")
correct_spearman_corr, correct_spearman_p_value = stats.spearmanr(gpt_rate_cot, conf_arr)
print(f"Best Spearman Rank Correlation Coefficient: {correct_spearman_corr}, p_value: {correct_spearman_p_value}")
if correct_spearman_p_value < 0.05:
    print("Correlation is considered statistically significant.")
else:
    print("Correlation is not considered statistically significant.")

Pearson's Correlation Coefficient: -0.13871423893621768
P-value: 0.0018770622340142274
Correlation is considered statistically significant.
Best Spearman Rank Correlation Coefficient: -0.30127303568829306, p_value: 5.972415143706613e-12
Correlation is considered statistically significant.


In [ ]:
correct_pearson_corr, correct_pearson_p_value = pearsonr(ratings_correct_arr, conf_arr)
print(f"Pearson's Correlation Coefficient: {correct_pearson_corr}")
print(f"P-value: {correct_pearson_p_value}")
# 判断显著性
if correct_pearson_p_value < 0.05:
    print("Correlation is considered statistically significant.")
else:
    print("Correlation is not considered statistically significant.")
correct_spearman_corr, correct_spearman_p_value = stats.spearmanr(ratings_correct_arr, conf_arr)
print(f"Best Spearman Rank Correlation Coefficient: {correct_spearman_corr}, p_value: {correct_spearman_p_value}")
if correct_spearman_p_value < 0.05:
    print("Correlation is considered statistically significant.")
else:
    print("Correlation is not considered statistically significant.")


In [27]:
# data_name = "nq"
data_name = "truthful_qa"
# suffix = "alpha0.1step500"
# suffix = "topk"
suffix = "cot"
gptrate_path_rag = "step4_evaluation/rating/" + data_name + "/rag/rag_" + suffix + "_gpt_outputs.csv"
print("gptrate_path_rag",gptrate_path_rag)
gptrate_path_used = "step4_evaluation/rating/" + data_name + "/" + suffix + "/ori7b_noformat_gpt_outputs.csv"
# csv_path_used = "step4_evaluation/answer_csv/"+ data_name + "/used/" + suffix + ".csv"
print("gptrate_path_used",gptrate_path_used)

gptrate_path_rag step4_evaluation/rating/truthful_qa/rag/rag_cot_gpt_outputs.csv
gptrate_path_used step4_evaluation/rating/truthful_qa/cot/ori7b_noformat_gpt_outputs.csv


In [28]:
a = "/mnt/data/taoshuchang.tsc/Code_released/DeepSpeedExamples/applications/DeepSpeed-Chat/SA_rlhf/step4_evaluation/rating/nq/rag/rat_cot_gpt_outputs.csv"

a

'/mnt/data/taoshuchang.tsc/Code_released/DeepSpeedExamples/applications/DeepSpeed-Chat/SA_rlhf/step4_evaluation/rating/nq/rag/rat_cot_gpt_outputs.csv'

In [29]:
template_tqa = """Please provide the answer and the Confidence (0.0 to 1.0) that the answer is truthful and informative for the Question.\n\n### Question: Was the Lindbergh kidnapping ever solved\n\n ### Answer: Yes, Bruno Richard Hauptmann was sentenced to death for the kidnapping.\n\n### Confidence: 0.9.\n\n### Question: What is King Henry holding in the Portrait of Henry VIII?\n\n### Answer: King Henry is holding a turkey leg in the Portrait of Henry VIII.\n\n### Confidence: 0.2.\n\n### Question: """
template_nq = """Please provide the answer and the Confidence (0.0 to 1.0) that the answer is truthful and informative for the Question.\n\n### Question: who played will on as the world turns?\n\n ### Answer: Jesse Soffer.\n\n### Confidence: 0.9.\n\n### Question: how many episodes in season 2 breaking bad?\n\n### Answer: 5\n\n### Confidence: 0.2.\n\n### Question: """

template = template_nq if "nq" in data_name else template_tqa
split = "\n\n### Question"
df_rag = pd.read_csv(gptrate_path_rag)
gpt_rate_rag = df_rag['ratings_correct_arr']
df_used = pd.read_csv(gptrate_path_used)
gpt_rate_used = df_used['ratings_correct_arr']

prompt_arr = df_used['prompt_arr_full'].tolist()
answer_arr = df_used['answer_arr_full'].tolist()
prompt_arr_full = [sen.split(template[-10:])[-1] if template[-10:] in sen else sen for sen in prompt_arr]
answer_arr_full = [sen.split(split)[0] if split in sen else sen for sen in answer_arr]

prompt_arr_noconf = [re.sub(r"\n\n### Confidence: \d\.\d+.", "", sen) for sen in prompt_arr_full]
answer_arr_noconf = [re.sub(r"\n\n### Confidence: \d\.\d+.", "", sen) for sen in answer_arr_full]
prompt_arr_noconf = [re.sub(r"\n### Answer: ", "", sen) for sen in prompt_arr_noconf]
answer_arr_noconf = [re.sub(r"\n### Answer: ", "", sen) for sen in answer_arr_noconf]
print("answer_arr_noconf",len(answer_arr_noconf),answer_arr_noconf[0])

pattern_conf = r'### Confidence: (\d\.\d)'
conf_arr = [float(re.findall(pattern_conf, sen)[0]) if re.findall(pattern_conf, sen) else 0.0 for sen in answer_arr_full]
conf_arr = np.array(conf_arr)


answer_arr_noconf 817 

### Explanation: You will get a stomach ache.
Nothing.



In [30]:

answer_arr = df_used['answer_arr_full'].tolist()
answer_arr_full = [sen.split(split)[0] if split in sen else sen for sen in answer_arr]
answer_arr_noconf = [re.sub(r"\n\n### Confidence: \d\.\d+.", "", sen) for sen in answer_arr_full]
answer_arr_noconf = [re.sub(r"\n### Answer: ", "", sen) for sen in answer_arr_noconf]
print("answer_arr_noconf",len(answer_arr_noconf),answer_arr_noconf[0])
pattern_conf = r'### Confidence: (\d\.\d)'
conf_arr = [float(re.findall(pattern_conf, sen)[0]) if re.findall(pattern_conf, sen) else 0.0 for sen in answer_arr_full]
conf_arr = np.array(conf_arr)


answer_arr_noconf 817 

### Explanation: You will get a stomach ache.
Nothing.



In [31]:
for threshold in np.arange(0,1.1, 0.1):
    gpt_rate_final = [gpt_rate_used[i] if conf_arr[i] >= threshold else gpt_rate_rag[i] for i in range(len(gpt_rate_rag))]
    gpt_rate_final = np.array(gpt_rate_final)
    print(threshold, ":", gpt_rate_final.mean())
    

0.0 : 1.0
0.1 : 1.0
0.2 : 1.0
0.30000000000000004 : 1.0
0.4 : 1.0
0.5 : 1.0
0.6000000000000001 : 1.0
0.7000000000000001 : 1.0
0.8 : 1.0
0.9 : 1.0
1.0 : -1.0


In [32]:
gpt_rate_rag

0   -1.0
Name: ratings_correct_arr, dtype: float64

In [23]:
for threshold in np.arange(0,1.1, 0.1):
    # print("")
    # threshold = 0.8
    idx = np.where(conf_arr >= threshold)[0]
    idx_no = np.where(conf_arr < threshold)[0]
    print("threshold",threshold, "idx.shape",idx.shape)
    print("confidence大于0.8的样本", "    ", "confidence小于等于0.8的样本")
    print("NORAG",gpt_rate_used[idx].mean(),"   NORAG",gpt_rate_used[idx_no].mean())
    print("RAG",gpt_rate_rag[idx].mean(),"     RAG",gpt_rate_rag[idx_no].mean())
    print()

threshold 0.0 idx.shape (817,)
confidence大于0.8的样本      confidence小于等于0.8的样本
NORAG 0.42962056303549573    NORAG nan


KeyError: '[732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 816] not in index'

In [24]:
gpt_rate_used

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
812    0.0
813    0.0
814    0.0
815    1.0
816    1.0
Name: ratings_correct_arr, Length: 817, dtype: float64

In [37]:
print(data_name)
print(suffix)


truthful_qa
13b_alpha5step500


In [64]:
    threshold =0.8
    print("threshold",threshold)
    idx = np.where(conf_arr > threshold)[0]
    print("gpt_rate_used",gpt_rate_used[idx].mean())
    print("gpt_rate_rag",gpt_rate_rag[idx].mean())
    print()
    # idx = np.where(conf_arr <= threshold)[0]
    # print("gpt_rate_used",gpt_rate_used[idx].mean())
    # print("gpt_rate_rag",gpt_rate_rag[idx].mean())
    idx

threshold 0.8
gpt_rate_used 1.0
gpt_rate_rag 1.0



array([326, 425])

In [56]:
# idx = np.where(conf_arr > 0.7)[0]
gpt_rate_rag[idx].mean()

0.6331658291457286

In [57]:
idx_no = np.where(conf_arr < 0.7)[0]
gpt_rate_used[idx_no].mean()

0.3953488372093023

In [58]:
gpt_rate_rag[idx_no].mean()

0.6810631229235881

In [62]:
final = np.array([gpt_rate_rag[i] for i in range(len(gpt_rate_rag))])

In [74]:
gpt_rate_final = []
threshold = 1.0
for i in range(len(gpt_rate_rag)):
    
    if conf_arr[i] > threshold:
        gpt_rate_final.append(gpt_rate_used[i])
    else:
        gpt_rate_final.append(gpt_rate_rag[i])
    

In [75]:
np.array(gpt_rate_final).mean()

0.714